# Packaging software in images

You can create a Docker image by either modifying an existing image inside a
container or defining and executing a build script called a Dockerfile.

## Building Docker images from a container

It’s easy to get started building images if you’re already familiar with using containers.
Remember, a union filesystem (UFS) mount provides a container’s filesystem. Any changes
that you make to the filesystem inside a container will be written as new layers owned
by the container that created them.

Before you work with real software, the next section details the typical workflow
with a “Hello, World” example.

### Packaging “Hello, World”

The basic workflow for building an image from a container includes three steps. 

1. First, you create a container from an existing image. You choose the image based on what
you want to be included with the new finished image and the tools you need to make
the changes.
2. The second step is to modify the filesystem of the container. These changes will be
written to a new layer of the container’s union filesystem. We’ll revisit the relationship
between images, layers, and repositories later in this chapter.

3. Once the changes have been made, the last step is to commit those changes. Then
you’ll be able to create new containers from the resulting image. 

With these steps in mind, work through the following commands to create a new image
named hw_image:

Modifies file in container:

    docker container run --name hw_container \
            ubuntu:latest \
            touch /HelloWorld


    docker ps -a

Commits change to new image:

    docker image ls
    docker container commit hw_container hw_image
    docker image ls

Removes changed container

    docker container rm -vf hw_container

Examines file in new container

    docker container run --rm \
        hw_image \
        ls -l /HelloWorld

If that seems stunningly simple, you should know that it does become a bit more
nuanced as the images you produce become more sophisticated, but the basic steps
will always be the same. Now that you have an idea of the workflow, you should try to
build a new image with real software. In this case, you’ll be packaging a program
called Git.

## Primer: NodeJS App

Naredimo mapo `simpleweb`

- Naredimo package.json in index.js

```json
{
  "dependencies": {
    "express": "*"
  },
  "scripts": {
    "start": "node index.js"
  }
}

```

```javascript
const express = require("express");

const app = express();

app.get("/", (req, res) => {
  res.send("How are you doing");
});

app.listen(8080, () => {
  console.log("Listening on port 8080");
});



```

Začnemo delat Dockerfile

```Dockerfile
# Specify a base image
FROM node:alpine

WORKDIR /usr/app

# Install some depenendencies
COPY ./package.json ./
RUN npm install
COPY ./ ./

EXPOSE 8080

# Default command
ENTRYPOINT ["npm", "start"]
```

- pokažemo kako dobimo node:alpine sliko, pa tage razložimo
- napišemo ukaz za zagon
- pokažemo najprej brez kopiranja datotek, pa vidimo da manjkajo
- workdir -> da ne bo v rootu, da ne overwrita kakšne druge sistemske mape
- dva copy-ja da ni potrebno rebildat vsega vsakič ko spremenimo aplikaicjo, če ne spremnimo odvisnosti

https://docs.docker.com/engine/reference/builder/#understand-how-cmd-and-entrypoint-interact

Zgradimo sliko:
    
    docker build -t leon11sj/simpleweb . 
    docker run --rm leon11sj/simpleweb

Zaženemo brskalnik, probamo ugotoviti zakaj ne dela?

Ponovimo z port mappingom

    docker run --rm -p 80:8080 leon11sj/simpleweb

## Building images automatically with Dockerfiles

**A Dockerfile is a text file that defines a Docker image. You’ll use a Dockerfile to create your own custom Docker image, in other words to define your custom environment to be used in a Docker container.**

1. The Dockerfile is a text file that (mostly) contains the instructions that you would execute on the command line to create an image.
2. A Dockerfile is a step by step set of instructions.
3. Docker provides a set of standard instructions to be used in the Dockerfile, like FROM, COPY, RUN, ENV, EXPOSE, CMD just to name a few basic ones.
4. Docker will build a Docker image automatically by reading these instructions from the Dockerfile.

This also implies that understanding Dockerfile instructions is not enough to create your Dockerfile, because **you need to also understand the context of the technology** you are building for.

A Dockerfile is a text file that contains instructions for building an image. The
Docker image builder executes the Dockerfile from top to bottom, and the instructions
can configure or change anything about an image. Building images from
Dockerfiles makes tasks like adding files to a container from your computer simple
one-line instructions. Dockerfiles are the most common way to describe how to
build a Docker image.

Once an
image’s build is defined in code, it is simple to track changes in version control, share
with team members, optimize, and secure.

### Packaging Git with a Dockerfile

You
should recognize many of the details and advantages of working with a Dockerfile as
we translate the image build process from manual operations to code.
First, create a new directory, and from that directory create a new file with your
favorite text editor. Name the new file Dockerfile. Write the following five lines and
then save the file:

> Ustvarimo datoteko Dockerfile

    # An example Dockerfile for installing Git on Ubuntu
    FROM ubuntu:latest
    LABEL maintainer="leon@ltfe.org"
    RUN apt-get update && apt-get install -y git
    ENTRYPOINT ["git"]

Before dissecting this example, build a new image from it with the docker image
build command from the same directory containing the Dockerfile and tag the
image with auto:

    docker image build --tag ubuntu-git:auto .

This outputs several lines about steps and output from apt-get, and will finally display
a message like this:

    Successfully built cc63aeb7a5a2
    Successfully tagged ubuntu-git:auto

Running this command starts the build process. When it’s completed, you should
have a brand-new image that you can test. View the list of all your ubuntu-git images
and test the newest one with this command:

    docker image ls

Now you can run a Git command using the new image:

    docker container run --rm ubuntu-git:auto

These commands demonstrate that the image you built with the Dockerfile works and
is functionally equivalent to the one you built by hand. Examine what you did to
accomplish this:

First, you created a Dockerfile with four instructions:
- `FROM ubuntu:latest` — Tells Docker to start from the latest Ubuntu image just as
you did when creating the image manually.
-  `LABEL maintainer` — Sets the maintainer name and email for the image. Providing
this information helps people know whom to contact if there’s a problem
with the image. This was accomplished earlier when you invoked commit.
- `RUN apt-get update && apt-get install -y git` — Tells the builder to run the
provided commands to install Git.
- `ENTRYPOINT ["git"]` — Sets the entrypoint for the image to git.

Dockerfiles, like most scripts, can include comments. Any line beginning with a # will
be ignored by the builder.

It’s important for Dockerfiles of any complexity to be **welldocumented**.
In addition to improving Dockerfile maintainability, comments help
people audit images that they’re considering for adoption and spread best practices.

The only special rule about Dockerfiles is that the **first instruction must be FROM**. If
you’re starting from an empty image and your software has no dependencies, or you’ll
provide all the dependencies, then you can start from a special empty repository
named scratch.

After you saved the Dockerfile, **you started the build process** by invoking the `docker image build` command. The command had one flag set and one argument. The `--tag`
flag (or -t for short) specifies the **full repository designation that you want to use for the
resulting image.** In this case, you used ubuntu-git:auto. The argument that you
included at the end was a single period. That argument told the builder the location of
the Dockerfile. The period told it to look for the file in the current directory.

The docker image build command has another flag, `--file` (or -f for short), that
lets you **set the name of the Dockerfile**. Dockerfile is the default, but with this flag
you could tell the builder to look for a file named BuildScript or release-image.df.
This flag sets only the name of the file, not the location. That must always be specified
in the location argument.

The builder works by automating the same tasks that you’d use to create images by
hand. Each instruction triggers the creation of a new container with the specified
modification. After the modification has been made, **the builder commits the layer
and moves on to the next instruction and container created from the fresh layer**.

The builder validated that the image specified by the FROM instruction was installed
as the first step of the build. If it were not, Docker would have automatically tried to
pull the image.

**Installing software packages is one of the most common use cases
for the RUN instruction**. You should explicitly install each software package needed by
your container to ensure that it is available when needed.

If you’re not interested in reams of build process output, you can invoke the
docker image build command with the --quiet or -q flag. Running in quiet mode
will suppress all output from the build process and management of intermediate containers.
The only output of the build process in quiet mode is the resulting image ID,
which looks like this:

    sha256:e397ecfd576c83a1e49875477dcac50071e1c71f76f1d0c8d371ac74d97bbc90

Although this third step to install Git usually takes much longer to complete, you can
see the instruction and input as well as the ID of the container where the command
was run and the ID of the resulting layer. Finally, the ENTRYPOINT instruction performs
all the same steps, and the output is similarly unsurprising:
    
    Step 4/4 : ENTRYPOINT ["git"]
    ---> Running in 6151803c388a
    Removing intermediate container 6151803c388a
    ---> e397ecfd576c
    Successfully built e397ecfd576c
    Successfully tagged ubuntu-git:auto

**A new layer is being added to the resulting image after each step in the build.**
Although this means you could potentially branch on any of these steps, the more
important implication is that the builder can aggressively cache the results of each
step. If a problem with the build script occurs after several other steps, the builder can
restart from the same position after the problem has been fixed.

Add this line to the end of your Dockerfile:
    
    RUN This will not work

Then run the build again:
    
    docker image build --tag ubuntu-git:auto .

> Note use of cache.

Steps 1 through 4 were skipped because they were already built during your last build.
Step 5 failed because there’s no program with the name This in the container. The
**container output was valuable in this case because the error message informs you
about the specific problem with the Dockerfile**. If you fix the problem, the same steps
will be skipped again, and the build will succeed, resulting in output like Successfully
built d7a8ee0cebd4.

> Popravimo še enkrat, da deluje.

> What is more important that with every step in the build process Docker will create an intermediary image for the specific step.

The **use of caching during the build can save time if the build includes downloading
material, compiling programs, or anything else that is time-intensive.** If you need a
full rebuild, you can use the `--no-cache` flag on docker image build to disable the
use of the cache. Make sure you’re disabling the cache only when required because it
will place much more strain on upstream source systems and image-building systems.

> Please note that each layer only stores the differences compared to the underlying layer. The right interpretation is that docker images and docker images -a display the size of the image including the size of parent images.

### A Dockerfile primer

Dockerfiles are expressive and easy to understand because of their terse syntax that
allows for comments. You **can keep track of changes to Dockerfiles with any version control
system**. Maintaining multiple versions of an image is as simple as maintaining multiple Dockerfiles.

The Dockerfile build process itself uses extensive caching to aid
rapid development and iteration. The builds are traceable and reproducible. They
integrate easily with existing build systems and many continuous integration tools.
With all these reasons to prefer Dockerfile builds to handmade images, it’s important
to learn how to write them.

#### .dockerignore

The directory where you issue the docker build command is called the build context. Docker will send all of the files and directories in your build directory to the Docker daemon as part of the build context. If you have stuff in your directory that is not needed by your build, you’ll have an unnecessarily larger build context that results in a larger image size.

You can remedy this situation by adding a .dockerignore file that works similarly to .gitignore. You can specify the list of folders and files that should be ignored in the build context.

One of the best reasons to use Dockerfile builds is that they simplify copying files from your computer into an image. But it’s not always appropriate for certain files to be copied to images. The first thing to do when starting a new project is to define which files should never be copied into any images. You can do this in a file called `.dockerignore`. In this example, you’ll create three Dockerfiles, and none needs to be copied into the resulting images.

    .dockerignore
    Dockerfile

Save and close the file when you’re finished. This will prevent the .dockerignore file, or files named Dockerfile, from ever being copied into an image during a build. With that bit of accounting finished, you can begin working on the base image.

#### FROM

For beginners it’s enough to understand that **every Dockerfile must start with the FROM instruction in the form of FROM `<image>[:tag]`.** This will set the base image for your Dockerfile, which means that subsequent instructions will be applied to this base image.

The tag value is optional, if **you don’t specify the tag Docker will use the tag latest and will try and use or pull the latest version of the base image during build.**


```Dockerfile

# pull official base image
FROM python:3.8.1-slim-buster
    
   ```

#### ENV

Five new instructions are introduced in this Dockerfile. The first new instruction is
ENV. ENV sets environment variables for an image, similar to the --env flag on docker
container run or docker container create. In this case, a single ENV instruction is
used to set three distinct environment variables. That could have been accomplished
with three subsequent ENV instructions, though doing so would result in the creation
of three layers. You can keep instructions easy to read by using a backslash to escape
the newline character (just as in shell scripting).

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 
```    

>  When you create a Dockerfile, you need to keep in mind that each Dockerfile **instruction will result in a new layer being created**. Instructions should be combined whenever possible because the builder won’t perform any optimization.

Environment variables declared in the Dockerfile are made available to the resulting image but can be used in other Dockerfile instructions as substitutions. In this Dockerfile, the environment variable VERSION was used as a substitution in the next new instruction, LABEL.

#### LABEL

The LABEL instruction is used to define key/value pairs that are recorded as additional
metadata for an image or container. This mirrors the --label flag on docker run and
docker create. Like the ENV instruction before it, multiple labels can and should be
set with a single instruction. In this case, the value of the VERSION environment variable
was substituted for the value of the base.version label. By using an environment
variable in this way, the value of VERSION will be available to processes running inside a
container as well as recorded to an appropriate label. **This increases maintainability of
the Dockerfile because it’s more difficult to make inconsistent changes when the valueis
set in a single location**.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"


 ```   

**WORKDIR**

The result of the WORKDIR instruction will be an image with the default working directory
set to /app. Setting WORKDIR to a location that doesn’t exist will create that location
just as it would with the command-line option.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT
```   

#### EXPOSE

An important instruction to inform your users about the ports your application is listening on. EXPOSE will not publish the port, you need to use docker run -p... to do that when you start the container.

Last, the EXPOSE command creates a layer that opens TCP port 5000.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000
``` 

#### RUN

RUN will execute commands, so it’s one of the most-used instructions. I would like to highlight two points:
1. You’ll use a lot of apt-get type of commands to add new packages to your image. It’s always advisable to put apt-get update and apt-get install commands on the same line. This is important because of layer caching. Having these on two separate lines would mean that if you add a new package to your install list, the layer with apt-get update will not be invalidated in the layer cache and you might end up in a mess. 
2. RUN has two forms; `RUN <command>` (called shell form) and `RUN ["executable", "param1", "param2"]` called exec form. Please note that `RUN <command>` will invoke a shell automatically (/bin/sh -c by default), while the exec form will not invoke a command shell.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip

``` 

#### COPY vs ADD

Both ADD and COPY are designed to add directories and files to your Docker image in the form of `ADD <src>... <dest>` or `COPY <src>... <dest>`. Most resources, including myself, **suggest to use COPY**.

The reason behind this is that ADD has extra features compared to COPY that make ADD more unpredictable and a bit over-designed. ADD can pull files from url sources, which COPY cannot. ADD can also extract compressed files assuming it can recognize and handle the format. You cannot extract archives with COPY.

The ADD instruction was added to Docker first, and COPY was added later to provide a straightforward, rock solid solution for copying files and directories into your container’s file system.

If you want to pull files from the web into your image I would suggest to use RUN and curl and uncompress your files with RUN and commands you would use on the command line.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

``` 

#### USER

Don’t run your stuff as root, be humble, use the USER instruction to specify the user. This user will be used to run any subsequent RUN, CMD AND ENDPOINT instructions in your Dockerfile.

By default, Docker runs container processes as root inside of a container. This is a bad practice since attackers can gain root access to the Docker host if they manage to break out of the container. If you're root in the container, you'll be root on the host.

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# create the app user
RUN groupadd -r -g 2200 app && \
    useradd -rM -g app -u 2200 app

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

# chown all the files to the app user
RUN chown -R app:app $APPROOT

# change to the app user
USER app
``` 

#### ENTRYPOINT & CMD

The ENTRYPOINT instruction has two forms: the **shell** form and an **exec** form. The
shell form looks like a shell command with whitespace-delimited arguments. The exec
form is a string array in which the first value is the command to execute and the
remaining values are arguments. A command specified using the shell form would be
executed as an argument to the default shell. Specifically, the command used in this
Dockerfile will be executed as /bin/sh –c 'exec ./mailer.sh' at runtime. Most
importantly, if the shell form is used for ENTRYPOINT, all other arguments provided by
the CMD instruction or at runtime as extra arguments to docker container run will be
ignored. This makes the shell form of ENTRYPOINT less flexible.

Naša končna slika:

```Dockerfile
# pull official base image
FROM python:3.8.1-slim-buster

LABEL maintainer="leon@ltfe.org"

# set environment variables
ENV APPROOT="/home/app/web" \
    PYTHONDONTWRITEBYTECODE=1 \
    PYTHONUNBUFFERED=1 \
    VERSION="0.1" 

LABEL base.name="Smart API" \
      base.version="${VERSION}"

# create the app user
RUN groupadd -r -g 2200 app && \
    useradd -rM -g app -u 2200 app

# set work directory
WORKDIR $APPROOT

EXPOSE 5000

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
RUN pip install --upgrade pip
COPY ./requirements.txt $APPROOT/requirements.txt
RUN pip install -r requirements.txt

# copy project
COPY . $APPROOT

# chown all the files to the app user
RUN chown -R app:app $APPROOT

# za testne namene
RUN apt-get install -y procps

# change to the app user
USER app


ENTRYPOINT ["python", "main.py"]
CMD ["-B"]
``` 

Both CMD and ENTRYPOINT instructions define what command gets executed when running a container. There are few rules that describe their co-operation.
1. Dockerfile should specify at least one of `CMD` or `ENTRYPOINT` commands.
2. `ENTRYPOINT` should be defined when using the container as an executable.
3. `CMD` should be used as a way of defining default arguments for an `ENTRYPOINT` command or for executing an ad-hoc command in a container.
4. `CMD` will be overridden when running the container with alternative arguments.

The table below shows what command is executed for different ENTRYPOINT / CMD combinations:

<table>
  <thead>
    <tr>
      <th style="text-align: left">&nbsp;</th>
      <th style="text-align: left">No ENTRYPOINT</th>
      <th style="text-align: left">ENTRYPOINT exec_entry p1_entry</th>
      <th style="text-align: left">ENTRYPOINT [“exec_entry”, “p1_entry”]</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left"><strong>No CMD</strong></td>
      <td style="text-align: left"><em>error, not allowed</em></td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD [“exec_cmd”, “p1_cmd”]</strong></td>
      <td style="text-align: left">exec_cmd p1_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry exec_cmd p1_cmd</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD [“p1_cmd”, “p2_cmd”]</strong></td>
      <td style="text-align: left">p1_cmd p2_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry p1_cmd p2_cmd</td>
    </tr>
    <tr>
      <td style="text-align: left"><strong>CMD exec_cmd p1_cmd</strong></td>
      <td style="text-align: left">/bin/sh -c exec_cmd p1_cmd</td>
      <td style="text-align: left">/bin/sh -c exec_entry p1_entry</td>
      <td style="text-align: left">exec_entry p1_entry /bin/sh -c exec_cmd p1_cmd</td>
    </tr>
  </tbody>
</table>

> If CMD is defined from the base image, setting ENTRYPOINT will reset CMD to an empty value. In this scenario, CMD must be defined in the current image to have a value.


The `exec form`, which is the preferred form:
    
    ENTRYPOINT ["executable", "param1", "param2"]

The `shell form`:

    ENTRYPOINT command param1 param2

> TIP:  **Using the exec (or string array) form wherever possible is the best practice.**
At a minimum, a Dockerfile should be consistent and avoid mixing styles.
This will make your Dockerfiles more readable and ensure that instructions
behave as you’d expect without detailed understanding of their nuances.

Command line arguments to docker run <image> will be appended after all elements in an exec form ENTRYPOINT, and will override all elements specified using CMD. This allows arguments to be passed to the entry point, i.e., docker run <image> -d will pass the -d argument to the entry point. You can override the ENTRYPOINT instruction using the docker run --entrypoint flag.

    docker image build -t leon11sj/entrypointcmd:v1 -f Dockerfile_01 .

    docker run -it --rm leon11sj/entrypointcmd:v1

    docker run -it --rm leon11sj/entrypointcmd:v1 -la

    docker run -it --rm --entrypoint top leon11sj/entrypointcmd:v1

    docker run -it --rm --entrypoint ps leon11sj/entrypointcmd:v1 aux

The shell form prevents any CMD or run command line arguments from being used, but has the disadvantage that your ENTRYPOINT will be started as a subcommand of /bin/sh -c, which does not pass signals. This means that the executable will not be the container’s PID 1 - and will not receive Unix signals - so your executable will not receive a SIGTERM from docker stop <container>.

Only the last ENTRYPOINT instruction in the Dockerfile will have an effect.

    ENTRYPOINT ["python", "main.py"]

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run -d --name test leon11sj/smart-api:v0.1
    docker exec -it test ps x
    docker rm -f test

Druga opcija v shell obliki

    ENTRYPOINT python main.py

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run -d --name test leon11sj/smart-api:v0.1
    docker exec -it test ps x
    docker rm -f test

<hr>

CMD instruction allows you to set a default command, which will be executed only when you run container without specifying a command. If Docker container runs with a command, the default command will be ignored. If Dockerfile has more than one CMD instruction, all but last CMD instructions are ignored.

CMD has three forms:
- `CMD ["executable","param1","param2"]` (exec form, preferred)
- `CMD ["param1","param2"]` (sets additional default parameters for ENTRYPOINT in exec form)
- `CMD command param1 param2` (shell form)

<hr>

**The difference is ENTRYPOINT command and parameters are not ignored when Docker container runs with command line parameters.**

Exec form of ENTRYPOINT allows you to set commands and parameters and then use either form of CMD to set additional parameters that are more likely to be changed. **ENTRYPOINT arguments are always used, while CMD ones can be overwritten by command line arguments provided when Docker container runs**. For example, the following snippet in Dockerfile

    docker image build -t leon11sj/entrypointcmd:v3 -f Dockerfile_03 .

    docker run -it --rm leon11sj/entrypointcmd:v3

    docker run -it --rm leon11sj/entrypointcmd:v3 John

> Shell form of ENTRYPOINT ignores any CMD or docker run command line arguments.

<hr>

**Zaključki**:
- Prefer ENTRYPOINT to CMD when building executable Docker image and you need a command always to be executed. Additionally use CMD if you need to provide extra default arguments that could be overwritten from command line when docker container runs.
- Choose CMD if you need to provide a default command and/or arguments that can be overwritten from command line when docker container runs.

### Building the file

Put it all together by running the docker image build command from the directory where the mailer-base file is located. The -f flag tells the builder which filename to use as input:

    docker image build -t leon11sj/smart-api:v0.1 -f Dockerfile.prod .
    docker run --rm --name smart-api -p 80:5000 leon11sj/smart-api:v0.1

> **Naming Dockerfiles**: The default and most common name for a Dockerfile is Dockerfile. However, Dockerfiles
can be named anything because they are simple text files and the build command
accepts any filename you tell it. Some people name their Dockerfiles with an
extension such as .df so that they can easily define builds for multiple images in a
single project directory (for example, app-build.df, app-runtime.df, and app-debugtools.
df). A file extension also makes it easy to activate Dockerfile support in editors.

## Best practices for writing Dockerfiles

### Start your Dockerfile with the steps that are least likely to change

This is easier said than done. Anyway, your image will stabilize after a while and changes will be less likely. The best practice is to structure your Dockerfile according to the following:
1. Install tools that are needed to build your application.
2. Install dependencies, libraries and packages.
3. Build your application.

### Clean up your Dockerfile

Always review your steps in the Dockerfile and only keep the minimum set of steps that are needed by your application. Always remove unnecessary components.

### Containers should be ephemeral

This would belong to generic Docker guidelines, but it’s never enough to stress this point. It is your best interest to design and build Docker images that can be destroyed and recreated/replaced automatically or with minimal configuration.

Which means that you should create Dockerfiles that define stateless images. Any state, should be kept outside of your containers.

### One container should have one concern

Think of containers as entities that take responsibility for one aspect of your project. So design your application in a way that your web server, database, in-memory cache and other components have their own dedicated containers.

You’ll see the benefits of such a design when scaling your app horizontally. We’ll look into interoperability of containers and container networking in a future tutorial.

## Docker images

Use the command docker images in your terminal to list the images you currently have on your computer. Remember that images are stored on your computer once you pull them from a registry like the Docker store, or once you build them on your computer.

    docker image ls
    docker images

Please note that I usually don’t keep all the images that I use, I try to keep everything nice and clean, because images take up space.

It is worthwhile to check the image sizes in the picture. You’ll find that some images have a very small footprint, like the Alpine linux image, while more complex image take up a lot of space.



    docker images -a

### Dangling images

Our newly built image is ready to use, but the previous image that we built with curl is still hanging around and it does not have a proper tag or name right now. (You can check the image ids to see that this is the same image we built previously).

Docker calls such images dangling images.

You can use the following command to list dangling images:

    docker images --filter "dangling=true"

I personally don’t like it when images are just hanging around without a purpose, so here is how to remove them:

    docker rmi $(docker images -q --filter "dangling=true")

## Public and private software distribution

The methods included in the spectrum range from hosted registries such as Docker
Hub to totally custom distribution architectures or source-distribution methods. We
cover some of these subjects in more detail than others. We also place particular focus
on private registries because they provide the most balance between the two concerns.

### Publishing with hosted registries

As a reminder, Docker registries are **services that make repositories accessible to
Docker pull commands**. A registry hosts repositories. The simplest way to distribute
your images is by using **hosted registries**.

A hosted registry is a Docker registry service that’s owned and operated by a thirdparty
vendor. Docker Hub, Quay.io, and Google Container Registry are all examples
of hosted registry providers. By default, Docker publishes to Docker Hub. Docker
Hub and most other hosted registries provide both public and private registries, as
shown in figure 9.2.

#### Publishing with public repositories: “Hello World!” via Docker Hub

The simplest way to get started with public repositories on hosted registries is to push
a repository that you own to Docker Hub. To do so, all you need is a Docker Hub
account and an image to publish. If you haven’t done so already, sign up for a Docker
Hub account now.

Once you have your account, you need to create an image to publish. Create a new
Dockerfile named HelloWorld.df and add the following instructions:

    FROM busybox:latest
    CMD echo 'Hello World!'

Chapter 8 covers Dockerfile instructions. As a reminder, the FROM instruction tells the
Docker image builder which existing image to start the new image from. The CMD
instruction sets the default command for the new image. Containers created from this
image will display Hello World! and exit. Build your new image with the following
command:

    docker image build \
        -t leon11sj/hello-dockerfile \
        -f HelloWorld.df \
        .

Be sure to substitute your Docker Hub username in that command. Authorization to
access and modify repositories is based on the username portion of the repository
name on Docker Hub. If you create a repository with a username other than your
own, you won’t be able to publish it.

Publishing images on Docker Hub with the docker command-line tool requires
that you establish an authenticated session with that client. You can do that with the
login command:
    
    docker login

This command will prompt you for your username, email address, and password. Each
of those can be passed to the command as arguments using the --username, --email,
and --password flags. When you log in, the docker client maintains a map of your credentials
for the different registries that you authenticate with in a file. It will specifically
store your username and an authentication token, not your password.

You will be able to push your repository to the hosted registry after you’ve logged
in. Use the docker push command to do so:

    docker image push leon11sj/hello-dockerfile

The command output includes upload statuses and the resulting repository content
digest. The push operation will create the repository on the remote registry, upload
each of the new layers, and then create the appropriate tags.

Your public repository will be available to the world as soon as the push operation
is completed. Verify that this is the case by searching for your username and your new
repository. For example, use the following command to find the example owned by
the dockerinaction user:
    
    docker search leon11sj

Replace the dockerinaction username with your own to find your new repository on
Docker Hub. You can also log in to the Docker Hub website and view your repositories
to find and modify your new repository.
Having distributed your first image with Docker Hub, you should consider how
this method measures up to the selection criteria;

Public repositories on hosted registries are the best choice for owners of open source
projects or people who are just getting started with Docker. People should still be
skeptical of software that they download and run from the internet, so public repositories
that don’t expose their sources can be difficult for some users to trust. Hosted
(trusted) builds solve this problem to a certain extent.

### Private hosted repositories

Private repositories are similar to public repositories from an operational and product
perspective. Most registry providers offer both options, and any differences in provisioning
through their websites will be minimal. Because the Docker registry API
makes no distinction between the two types of repositories, registry providers that
offer both generally require you to provision private registries through their website,
app, or API.

The tools for working with private repositories are identical to those for working
with public repositories, with one exception. Before you can use docker image pull or
docker container run to install an image from a private repository, you need to
authenticate with the registry where the repository is hosted. To do so, you use the
docker login command just as you would if you were using docker image push to
upload an image.

### Introducing private registries

When you have a hard requirement on availability control, longevity control, or
secrecy, then running a private registry may be your best option. In doing so, you gain
control without sacrificing interoperability with Docker pull and push mechanisms or
adding to the learning curve for your environment. People can interact with a private
registry exactly as they would with a hosted registry.

Many free and commercially supported software packages are available for running a
Docker image registry. If your organization has a commercial artifact repository for
operating system or application software packages, it probably supports the Docker
image registry API. A simple option for running a nonproduction image registry is to
use Docker’s registry software. The Docker registry, called Distribution, is open source
software and distributed under the Apache 2 license. The availability of this software
and permissive license keep the engineering cost of running your own registry low. Figure
9.3 illustrates that private registries fall in the middle of the distribution spectrum.

The biggest trade-off when going from hosted registries to private registries is gaining
flexibility and control while requiring greater depth and breadth of engineering experience
to build and maintain the solution. Docker image registries often consume
large amounts of storage, so be sure to account for that in your analysis. The remainder
of this section covers what you need in order to implement all but the most complicated
registry deployment designs and highlights opportunities for customization
in your environment.